Übung:  09-2 Gateway API
-------------------

![](demo/images/Microservices-REST.png)

Quelle: Buch Microservices Rezepte
- - -

Das Beispiel besteht aus drei Microservices: **Order**, **Customer** und **Catalog**. 

**Order** nutzt **Catalog** und **Customer** mit der REST-Schnittstelle. Ausserdem bietet jeder Microservice einige HTML-Seiten an.

Statt des Apache-Webservers, der als [Reverse Proxy](https://github.com/ewolff/microservice-kubernetes/blob/master/microservice-kubernetes-demo/apache/000-default.conf) konfiguriert ist, wird die Kubernetes Ressource Ingress verwendet.

Das Gateway API
---

Gateway API ist ein offizielles Kubernetes-Projekt, das sich auf L4- und L7-Routing in Kubernetes konzentriert. Dieses Projekt stellt die nächste Generation von Kubernetes Ingress-, Load Balancing- und Service Mesh-APIs dar. Es war von Anfang an generisch, ausdrucksstark und rollenorientiert konzipiert.

Das Gesamtressourcenmodell konzentriert sich auf drei separate [Personas](https://gateway-api.sigs.k8s.io/concepts/roles-and-personas) (Infrastructur, Cluster, Application) und entsprechende Ressourcen, die von ihnen verwaltet werden sollen:

![](https://gateway-api.sigs.k8s.io/images/resource-model.png)

Quelle: [Gateway API](https://gateway-api.sigs.k8s.io/)
- - -

Da die Gateway Ressourcen nicht Bestandteil vom Standard Kubernetes sind, sind zuerst die [Ressourcen zu erstellen](https://gateway-api.sigs.k8s.io/guides/#installing-a-gateway-controller):

In [ ]:
! kubectl apply -f https://github.com/kubernetes-sigs/gateway-api/releases/download/v1.0.0/standard-install.yaml

Wir Starten die Microservices wie zuvor. Dabei ist zu Beachten, dass die Persona **"Application Developers"** für die Services zuständig sind.

In [ ]:
%%bash
kubectl create namespace ms-rest
# ! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/apache.yaml (obsolet!)
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/catalog.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/customer.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/order.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/postgres.yaml

Beim erstellen des Cluster haben die Persona **"Infrastructure Provider"** die GatewayClass erstellt:

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: gateway.networking.k8s.io/v1
kind: GatewayClass
metadata:
  name: cluster-gateway
  namespace: ms-rest  
spec:
  controllerName: "ms-rest/gateway-controller"
%EOF%


Nach dem Starten erstellen die Persona **"Cluster Operators"** die Gateways:

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: gateway.networking.k8s.io/v1
kind: Gateway
metadata:
  name: ms-rest-gateway
spec:
  gatewayClassName: cluster-gateway
  listeners:
  - protocol: HTTP
    port: 80
    name: ms-rest-gw
%EOF%


Und als letzten Schritt die Persona **"Application Developer"** die HTTP Routen

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: gateway.networking.k8s.io/v1
kind: HTTPRoute
metadata:
  name: ms-rest-route
  namespace: ms-rest  
spec:
  parentRefs:
  - name: ms-rest-gateway
  rules:
  - matches:
    - path:
        type: PathPrefix
        value: /order/(.*)
    backendRefs:
    - name: order
      port: 8080
    - path:
        type: PathPrefix
        value: /customer
    backendRefs:
    - name: customer
      port: 8080  
    - path:
        type: PathPrefix
        value: /catalog
    backendRefs:
    - name: catalog
      port: 8080         
%EOF%


Überprüfen der erstellen Ressourcen

In [ ]:
! kubectl get pod,services,gatewayclass,gateway,httproute -n ms-rest

Alle Microservices sind jetzt mittels `https` und gleichem DNS bzw. IP-Adresse erreichbar.

In [ ]:
%%bash
export SERVER=http://$(cat ~/work/server-ip)
echo "Kunden    : ${SERVER}/customer"
echo "Produkte  : ${SERVER}/catalog"
echo "Bestellung: ${SERVER}/order/index.html"

***
### Ingress Service (nginx Server)

In der aktuellen Umgebung übernimmt ein nginx Server die Ingress Funktionalität. Dieser Server läuft als Pods im Namespace ingress-nginx.

Von dem nginx Server können wir uns die Konfigurationsdatei ausgeben:

In [ ]:
! kubectl exec daemonset/nginx-ingress-microk8s-controller -n ingress -- cat /etc/nginx/nginx.conf | grep location

Zum Testen kann der `kubectl apply -f -` welche die Ingress Ressourcen anlegt, durch `kubectl delete -f -` ersetzt werden und dann der obige Befehl wieder ausgeführt werden.

Dann sollten die `location` Einträge für `customer`, `catalog` und `order` nicht mehr vorhanden sein.

- - -

Aufräumen

In [ ]:
! kubectl delete namespace ms-rest

***
Fragen
======

Beantwortet die Fragen einzeln oder in der Gruppe und vergleicht diese mit den Antworten


Was ein Ingress?
<details><summary>Antwort</summary><p>    
    Ein API-Objekt, das den externen Zugriff auf die Dienste in einem Cluster verwaltet, in der Regel mittels HTTP. 
    Grob entspricht der Ingress Dienst dem Reverse Proxy Muster. 
</p></details>

---

Was für eine Netzwerkstruktur verwendet Kubernetes?
<details><summary>Antwort</summary><p>    
     Kubernetes verwendet im Unterschied zu Docker eine flache Netzwerkstruktur. 
* Jeder Container kann mit jedem anderen ohne NAT kommunizieren.
* Alle Kubernetes Nodes können mit allen Containern (und in die andere Richtung) ohne NAT kommunizieren.
* Die IP, die ein Container von sich selbst sieht, ist auch die, die jeder andere Node oder Container im Netz von ihm sieht.
</p></details>

---